# Overview

The Measurement Set v4 (MS v4) schema defines how correlated data (interferometer visibilities and single-dish spectra) can be represented in memory using datasets that consist of n-dimensional arrays labeled with coordinates and meta-information contained in attributes (see [foundational reading](https://xradio.readthedocs.io/en/latest/overview.html#Foundational-Reading)). The MS v4 implementation differs from the `MS v2` implementation in [`casacore`](https://github.com/casacore/casacore) primarily in its use of Python and off-the-shelf packages for data structures and data serialization, contrasting with `casacore's` bespoke C++ approach.

Reference documents consulted for the MS v4 schema design:

- [MeasurementSet definition version 2.0](https://casacore.github.io/casacore-notes/229.pdf)
- [MeasurementSet definition version 3.0β](https://casacore.github.io/casacore-notes/264.pdf)
- [MeasurementSet VLBI (Very Long Baseline Interferometry) extensions](https://casacore.github.io/casacore-notes/265.pdf)
- [CASA Ephemeris Data](https://casadocs.readthedocs.io/en/latest/notebooks/external-data.html#Ephemeris-Data)
- [ASDM (Astronomy Science Data Model): SDM Tables Short Description](https://drive.google.com/file/d/16a3g0GQxgcO7N_ZabfdtexQ8r2jRbYIS/view)

The current MS v4 schema focuses on offline processing capabilities and does not encompass all information present in the ASDM. However, its design allows for future expansion to incorporate additional data as needed (see [schema versioning](https://xradio.readthedocs.io/en/latest/overview.html#Schema-Versioning) section). It's important to note that MS v4 is not backward compatible with either MS v2 or MS v3, representing a significant evolution in the data model.

## Schema Layout

The MS v4 contains data for a single observation, spectral window, polarization setup, observation mode, and beam per antenna (though finer partitioning, such as splitting by scan, is allowed if desired). This structure simplifies the MS v4 data representation, enabling it to be stored as n-dimensional arrays with consistent shapes over time (rare baseline dropouts are handled by NaN padding). To process data from multiple MS v4s, we've introduced the concept of a Processing Set (`ps`), which is a collection of MS v4 files. Importantly, each MS v4 is fully self-describing. As shown in [Figure 1](#figure-1) (a simplified diagram; for full details, see the [Data Model Schema](file:///Users/jsteeb/Dropbox/xradio/docs/build/measurement_set/schema_and_api/measurement_set_schema.html)), the MS v4 is structured as a dataset (xds) of datasets comprising the `correlated_xds` along with `antenna_xds`, `pointing_xds`, `phase_calibration_xds`, `weather_xds`, `system_calibration_xds`, `gain_curve_xds`, and `phased_array_xds`, all stored in the attribute section. The `correlated_xds` contains the `VISIBILITY` (for interferometer data) or `SPECTRUM` (for single dish data), `UVW`, `WEIGHT`, and `FLAGS` data variables, along with info dictionaries in the attributes. The `field_and_source_xds` is specifically stored within the attributes of the `VISIBILITY`/`SPECTRUM` data variable.

<div style="text-align: center;">
    <figure id="figure-1" style="display: inline-block;">
        <img src="https://docs.google.com/drawings/d/e/2PACX-1vQVgjF5xNeIv8gpi2G3R8JXw2bNkVIUXdizIZluCGdnHc4z79ryW2fNUycJAd_CQh9sXLwdlx1oiAAX/pub?w=690&amp;h=510"
             alt="Processing Set and MS v4 schema layout."
             style="display: block; margin: auto;">
        <figcaption>Figure 1: Processing Set and MS v4 schema layout. Optional datasets are indicated by round brackets. Data variables are capitalized. The suffix '_xds' denotes an xarray dataset, while '_info' indicates dictionaries.</figcaption>
    </figure>
</div>

!!!!!! correlated_data set becomes part of ms_xds.

## Data Groups 

The `correlated_xds` can contain multiple copies of `VISIBILITY`/`SPECTRUM`, `UVW`, `WEIGHT`, and `FLAG` data variables. To maintain the relationship between a set of data variables, a `data_group` dictionary is used with fixed lowercase keys: 'correlated_data', 'uvw', 'weight', and 'flag'. 

The `data_groups` dictionary is stored as an attribute of the main dataset (`ms_xds`) and can contain one or more `data_group`s. Data variables can be shared between `data_group`s or be unique to a specific group. For example, you might have a 'base' group and an 'imaging' group:

```python
Example: ms_xds.attrs['data_groups'] = {
   'base': {'correlated_data': 'VISIBILITY', 'flag': 'FLAG', 'weight': 'WEIGHT', 'uvw': 'UVW'}, 
   'imaging': {'correlated_data': 'VISIBILITY_CORRECTED', 'flag': 'FLAG', 'weight': 'WEIGHT_IMAGING', 'uvw': 'UVW'}
}
```

The 'base' and 'imaging' `data_group`s share the same 'flag' and 'uvw' data variables but have different 'correlated_data' and 'weight' data variables. When creating new versions of data variables, they must contain the standard name followed by an underscore and a description chosen by the developer. For example, a new version of `VISIBILITY` could be `VISIBILITY_PHASE_SHIFTED`.

The `ms_xds` also allows for selecting a `data_group`. For example:

```Python
sub_ms_xds = ms_xds.sel(data_group='imaging')
```

## Use Cases

The MS v4 has been designed to satisfy the following use cases:

- Radio Interferometry 
- Single Dish Observations
- On-the-fly (OTF) Mosaic Observations 
- Ephemeris Observations 
- Heterogeneous Antenna VLBI 
- Phased Array Stations (PAS)
- Phased Array Feeds (PAF)

To ensure that these use cases are met, we have created stakeholder tests that convert MS v2 to MS v4 using the [convert_msv2_to_processing_set](https://xradio.readthedocs.io/en/latest/measurement_set/schema_and_api/measurement_set_api.html#xradio.measurement_set.convert_msv2_to_processing_set) function. These tests cover a wide selection of instruments:

### Interferometer Stakeholder Tests

| Dataset | Telescope | Mosaic | Ephemeris | VLBI | PAS | PAF |
|---------|-----------|:------:|:---------:|:----:|:---:|:---:|
| Antennae_North.cal.lsrk.split | ALMA | x |   |   | | |
| AA2-Mid-sim_00000 | Simulated SKA Mid |   |   |   | | |
| small_meerkat | MeerKAT |   |   |   | | |
| small_lofar | LOFAR |   |   |   | x | |
| global_vlbi_gg084b_reduced | VLBA+EVN |   |   | x | | |
| VLBA_TL016B_split.ms | VLBA |   |   | x | | |
| ngEHT_E17A10.0.bin0000.source0000_split | Simulated ngEHT |   |   | x | | |
| venus_ephem_test.ms | ALMA | x | x |   | | |
| ALMA_uid___A002_X1003af4_X75a3.split.avg.ms | ALMA | x | x |   | | |
| VLASS3.2.sb45755730.eb46170641.60480.16266136574.split.v6.ms | VLA (VLASS) | x |   |   | | |
| askap_59750_altaz_2settings | ASKAP |   |   |   | | x |
| askap_59754_altaz_2weights_0 | ASKAP |   |   |   | | x |
| askap_59754_altaz_2weights_15 | ASKAP |   |   |   | | x |
| askap_59755_eq_interleave_0 | ASKAP |   |   |   | | x |
| askap_59755_eq_interleave_15 | ASKAP |   |   |   | | x |

### Single Dish Stakeholder Tests

| Dataset | Telescope | Multi-Target | OFF Position | Ephemeris | Scan Pattern |
|---------|-----------|:------------:|:------------:|:---------:|:------------:|
| sdimaging | GBT |   | relative |   | raster |
| uid___A002_Xced5df_Xf9d9small | ALMA |   | horizontal |   | raster |
| uid___A002_Xe3a5fd_Xe38e.small | ALMA |   | relative | x | raster |
| uid___A002_X1015532_X1926f.small | ALMA | x | absolute |   | raster |
| uid___A002_Xae00c5_X2e6b.small | ALMA |   | relative | x | fast |

- OFF position: 
    - **absolute**: OFF position has its own field entry in the FIELD table.
    - **relative**: OFF is a fixed position specified as an offset coordinate relative to the ON position.
    - **horizontal**: OFF is also an offset position relative to the ON position but specified so that its elevation value is always close to the ON position (meaning that the OFF position is moving with time).
- Scan pattern: raster scan/fast scan.
    - **raster**: Scan is, as the name indicates, a raster pattern with a relatively long integration time (0.1-1 sec) and channelized spectral window.
    - **fast**: Scan is either a Lissajous or double-circle pattern with short integration time (1 msec) and single-channel spectral window.

## Translating between MS v2 and MS v4

| MS v2 table      | MS v4 Access Pattern                                              | Xarray Type                        |
|------------------|-------------------------------------------------------------------|------------------------------------|
| MAIN             | ms_xds                                                            | dataset                            |
| SCAN             | ms_xds                                                            | dataset                            |
| POLARIZATION     | ms_xds.polarization                                               | coord                              |
| SPECTRAL_WINDOW  | ms_xds.frequency                                                  | coord                              |
| DOPPLER          | ms_xds.frequency                                                  | coord                              |
| FIELD            | ms_xds.VISIBILITY/SPECTRUM.attrs.field_and_source_xds             | dataset                            |
| EPHEMERIDES      | ms_xds.VISIBILITY/SPECTRUM.attrs.field_and_source_xds             | dataset                            |
| SOURCE           | ms_xds.VISIBILITY/SPECTRUM.attrs.field_and_source_xds             | dataset                            |
| OBSERVATION      | ms_xds.attrs.observation_info                                     | dict                               |
| PROCESSOR        | ms_xds.attrs.processor_info                                       | dict                               |
| STATE            | ms_xds.attrs.state_info                                           | dataset                            |
| ANTENNA          | ms_xds.attrs.antenna_xds                                          | dataset                            |
| FEED             | ms_xds.attrs.antenna_xds                                          | dataset                            |
| PHASED_ARRAY     | ms_xds.phased_array_xds                                           | dataset                            |
| POINTING         | ms_xds.pointing_xds                                               | dataset                            |
| SYSCAL           | ms_xds.system_calibration_xds                                     | dataset                            |
| WEATHER          | ms_xds.weather_xds                                                | dataset                            |


## Summary of Key Changes

- !!!! Only contains 
- WEIGHT SPECTRUM is now WEIGHT !!!!
- Data is stored in Datasets of labeled n-dimensional arrays (called data variables) instead of tables.
- The table concept of rows has been replaced by relevant dimensions. For example, the VISIBILITY column in the MAIN table of MS v2 is now an n-dimensional array with dimensions time x baseline x frequency x polarization (row has been split into time x baseline).
- Most keys that used to be implicit numbered indices have been changed to descriptive names. This improves code readability, allows for sub-selecting data without reindexing, and enables easy data combination. For example, `antenna_name` is used instead of `antenna_id`.
- The concept of data description is deprecated and replaced by `spectral_window_name` and `polarization_setup`.
- Versioning of the VISIBILITY/SPECTRUM, WEIGHT, UVW, and FLAG data variables is done using [data groups](https://xradio.readthedocs.io/en/latest/measurement_set_overview.html#Data-Groups).
- The [JPL Horizons ephemerides](https://casadocs.readthedocs.io/en/latest/notebooks/external-data.html#Ephemeris-Data) was used to create the ephemeris schema instead of the MS v2.
- Field, source, and ephemeris data have been combined into a single dataset.
- Antenna and feed data have been combined into a single dataset (an MS v4 can only have one feed type per antenna).

## Depreciation List MS v2

The relevant string keys should be used instead.

- MAIN Table: BASELINE_REFERENCE, FIELD_ID, ANTENNA_ID, FEED1, FEED2, ANTENNA1, ANTENNA2, PROCESSOR_ID, DDI, FLAG_ROW (scan_number and baseline_id remain). 
- MAIN Table: 
- FIELD Table: FIELD_DELAY_CENTER, EPHEMERIS_ID.
- SOURCE Table: PROPER_MOTION, SOURCE_ID.
- ANTENNA Table: OFFSET 
- FEED Table: POL_RESPONSE, BEAM_OFFSET, POSITION. A beam model is under development which will
- POINTING: TARGET, SOURCE_OFFSET, ON_SOURCE, POINTING_OFFSET, TRACKING

## Planned future work
Beam models
Interferometer model


# Subpackage Layout

We can to support directly reading from WSU-ASDM, Zarr, Netcdf.
MSv2 we currently don't plan since querying is expensive.

Will eventually replace Python-casacore.
Asdm library does not exist yet.

Types of correlated datasets visibility (single dish) and spectrum (single dish)

Add link to API

<!-- # Link to google drawing: https://docs.google.com/drawings/d/1PMnElSu6YcMC9ovLOTARdYlLtZn7SJhUBdIHZYX52iU/edit?usp=sharing -->

Will eventually replace Python-casacore.
Asdm library does not exist yet.

<!-- # https://docs.google.com/drawings/d/1P9AI3D4VzPGw8O72dPBkz1iED8-ITM96anLeru0dvgM/edit?usp=sharing -->

<div style="text-align: center;">
    <figure id="figure-2" style="display: inline-block;">
        <img src="https://docs.google.com/drawings/d/e/2PACX-1vSxpmAjQ9Zmg2g5DqmwfiyE2i83Ci1EDeBuY7h8mHPYiokX-il9Omp2h7qmg5ZGbDPOJYUoCFHcL8E3/pub?w=885&h=706"
             alt="aaa."
             style="display: block; margin: auto;">
        <figcaption>Figure 2: xxxx.</figcaption>
    </figure>
</div>